In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def pair_pot(dist, params_ij, deriv=0):
    k_coul = 14.39965 # eV*Ang
    
    q1 = params_ij['q1']
    q2 = params_ij['q2']
    B = params_ij['B']
    rho = params_ij['rho']
    C = params_ij['C']
    
    if deriv==0:
        E   = +  k_coul*q1*q2/dist    + B       *np.exp(-dist/rho) -    C/dist**6
        result = E
    elif deriv==1:
        dE  = -  k_coul*q1*q2/dist**2 - B/rho   *np.exp(-dist/rho) + 6* C/dist**7
        result = dE
    elif deriv==2:
        d2E = +2*k_coul*q1*q2/dist**3 + B/rho**2*np.exp(-dist/rho) - 42*C/dist**8
        result = d2E
        
    return result
    
def force_const_term(cn, dist, params_ij):
    ang_bohr = 0.529177208
    eV_Ha = 27.211396
    
    dE = pair_pot(dist, params_ij, deriv=1)
    d2E = pair_pot(dist, params_ij, deriv=2)
    const = cn/6*(d2E + 2/dist*dE)
    const *= ang_bohr**2/eV_Ha # change to atomic units
    return const

def total_force_const(pair_names, cn_pairs, dist_pairs, params):
    force_const_pairs = []
    
    for pair in pair_names:
        cn = cn_pairs[pair]
        dist = dist_pairs[pair]
        params_ij = params[pair]
        print('\n -- ', pair, ' --')
        print('cn = ', cn)
        print('dist = ', dist)
        
        k_i = force_const_term(cn, dist, params_ij)
        force_const_pairs.append(k_i)
        
    force_const_pairs = np.array(force_const_pairs)
    
    print('k_pairs = ', force_const_pairs)
    print('rel k_pairs = ', force_const_pairs/force_const_pairs[-1])
    force_const = np.sum(force_const_pairs)
        
    return force_const

In [ ]:

def reverse_pair(pair, return_atoms=False):
    atoms = str.split(pair, '-')
    atoms_rev = atoms[::-1]
    pair_rev = atoms_rev[0]+'-'+atoms_rev[1]
    
    if return_atoms:
        return pair_rev, atoms[0], atoms[1]
    else:
        return pair_rev

def init_param_pairs(params):
    pair_names = [pair for pair in params]
    for pair in pair_names:
        pair_rev = reverse_pair(pair)
        
        if pair_rev != pair:
            params[pair_rev] = params[pair]
            
    
def init_cn_pairs(cn_pairs, comp):
    pair_names = [pair for pair in cn_pairs]
    for pair in pair_names:
        pair_rev, atom_i, atom_j = reverse_pair(
            pair, return_atoms=True)
        
        cn = cn_pairs[pair]
        if pair_rev != pair:
            cn_rev = cn*comp[atom_i]/comp[atom_j]
            cn_pairs[pair_rev] = cn_rev
    

In [ ]:
# Oganov2000 model
params_O2000 = {}
params_O2000['O-Mg']= {'q1':+1.9104, 'q2':-1.6049, 'B':1041.435, 
                      'rho':0.2866, 'C':0 }
params_O2000['O-Si']= { 'q1':+2.9043, 'q2':-1.6049, 'B':1137.028,
                      'rho':0.2827, 'C':0 }

params_O2000['O-O']= { 'q1':-1.6049, 'q2':-1.6049, 'B':2023.8,
                     'rho':0.2674, 'C':13.83 }

init_param_pairs(params_O2000)

# display(params_O2000)

# params_G2007 = {}

In [ ]:
# Guillot2007 model
eV_kJ_mol = 0.0103642688


params_G2007 = {}
params_G2007['O-Mg']= { 'q1':+0.945, 'q2':-0.945, 'B':3150507.4*eV_kJ_mol, 
                      'rho':0.178, 'C':2632.22*eV_kJ_mol }
params_G2007['O-Si']= { 'q1':+1.89, 'q2':-0.945, 'B':4853815.5*eV_kJ_mol,
                      'rho':0.161, 'C':4467.07*eV_kJ_mol }
params_G2007['O-Ti']= { 'q1':+1.89, 'q2':-0.945, 'B':4836495.0*eV_kJ_mol,
                      'rho':0.178, 'C':4467.07*eV_kJ_mol }
params_G2007['O-Al']= { 'q1':+1.4175, 'q2':-0.945, 'B':2753544.3*eV_kJ_mol,
                      'rho':0.172, 'C':3336.26*eV_kJ_mol }
params_G2007['O-Fe']= { 'q1':+0.945, 'q2':-0.945, 'B':1257488.6*eV_kJ_mol,
                      'rho':0.190, 'C':0.0 }
params_G2007['O-Ca']= { 'q1':+0.945, 'q2':-0.945, 'B':15019679.1*eV_kJ_mol,
                      'rho':0.178, 'C':4077.45*eV_kJ_mol }
params_G2007['O-Na']= { 'q1':+0.4725, 'q2':-0.945, 'B':11607587.5*eV_kJ_mol,
                      'rho':0.170, 'C':0.0 }
params_G2007['O-K']= { 'q1':+0.4725, 'q2':-0.945, 'B':220447.4*eV_kJ_mol,
                     'rho':0.290, 'C':0.0 }

params_G2007['O-O']= { 'q1':-0.945, 'q2':-0.945, 'B':870570.0*eV_kJ_mol,
                     'rho':0.265, 'C':8210.17*eV_kJ_mol }


init_param_pairs(params_G2007)
# display(params_G2007)

In [ ]:
################################
#  Bridgmanite composition
################################
pair_names_Brg = ['O-O','O-Mg','O-Si']
comp_Brg = {'Mg':1./5, 'Si':1./5, 'O':3./5}


################################
#  Simplified BSE composition
################################
pair_names_BSE = ['O-O','O-Ca','O-Na','O-Mg','O-Fe','O-Al','O-Si']
elems = ['Si', 'Al', 'Fe', 'Mg', 'Ca', 'Na', 
         'Ti', 'Cr', 'Mn', 'Ni', 'K', 'P', 'O']
elem_names_BSE = np.hstack((elems[:6], elems[-1]))
         
mol_elems_BSE = np.array([
    1.57718368e-01, 1.45564200e-02, 2.33867841e-02, 2.06614993e-01,
    1.04886125e-02, 2.02730603e-03, 4.16902047e-04, 1.06805185e-03,
    3.89325837e-04, 7.11119339e-04, 1.02954500e-04, 4.45573980e-05,
    5.82474606e-01])

mol_elems_BSE = np.hstack((mol_elems_BSE[:6], mol_elems_BSE[-1]))
mol_elems_BSE /= np.sum(mol_elems_BSE)


comp_BSE = {}
for elem, mol_elem in zip(elem_names_BSE, mol_elems_BSE):
    comp_BSE[elem] = mol_elem

comp_BSE




In [ ]:
def init_liq_struc(comp, cn_pairs, dist_pairs):
    liq_struc = {}
    
    init_cn_pairs(cn_pairs, comp)
    init_param_pairs(dist_pairs)
    
    liq_struc['comp'] = comp
    liq_struc['cn_pairs'] = cn_pairs
    liq_struc['dist_pairs'] = dist_pairs
    
    return liq_struc

In [ ]:
# Bridgmanite structure

struc_Brg = {}
struc_Brg['25'] = init_liq_struc(comp_Brg,
    {'Mg-O':6.8, 'Si-O':4.8, 'O-O':13},
    {'Mg-O':2.2, 'Si-O':1.755, 'O-O':2.7})

struc_Brg['135'] = init_liq_struc(comp_Brg,
    {'Mg-O':8.6, 'Si-O':6.0, 'O-O':13},
    {'Mg-O':2.0, 'Si-O':1.735, 'O-O':2.35})

# struc_Brg

In [ ]:
# BSE structure

struc_BSE = {}
struc_BSE['25'] = init_liq_struc(comp_BSE,
    {'Ca-O':9.0, 'Na-O':8.8, 'Mg-O':6.75, 
     'Fe-O':4.75, 'Al-O':5.75, 'Si-O':4.9, 'O-O':13},
    {'Ca-O':2.52, 'Na-O':2.5, 'Mg-O':2.2, 
     'Fe-O':1.95, 'Al-O':1.94, 'Si-O':1.76, 'O-O':2.7})


struc_BSE['135'] = init_liq_struc(comp_BSE,
    {'Ca-O':10.5, 'Na-O':9.8, 'Mg-O':8.4, 
     'Fe-O':6.0, 'Al-O':6.8, 'Si-O':6.1, 'O-O':13},
    {'Ca-O':2.2, 'Na-O':2.2, 'Mg-O':2.0, 
     'Fe-O':1.9, 'Al-O':1.86, 'Si-O':1.74, 'O-O':2.35})




In [ ]:
# BSE structure (diff between Mg/Si)

struc_BSE = {}
struc_BSE['25'] = init_liq_struc(comp_BSE,
    {'Ca-O':9.0, 'Na-O':8.8, 'Mg-O':6.75, 
     'Fe-O':4.75, 'Al-O':5.75, 'Si-O':4.9, 'O-O':13},
    {'Ca-O':2.52, 'Na-O':2.5, 'Mg-O':2.2, 
     'Fe-O':1.95, 'Al-O':1.94, 'Si-O':1.765, 'O-O':2.7})


struc_BSE['135'] = init_liq_struc(comp_BSE,
    {'Ca-O':10.5, 'Na-O':9.8, 'Mg-O':8.4, 
     'Fe-O':6.0, 'Al-O':6.8, 'Si-O':6.1, 'O-O':13},
    {'Ca-O':2.2, 'Na-O':2.2, 'Mg-O':2.0, 
     'Fe-O':1.9, 'Al-O':1.86, 'Si-O':1.745, 'O-O':2.35})




In [ ]:
def vib_freq(force_const, mass):
    """
    In atomic units
    """
    me_mp = 1836
    omega = np.sqrt(force_const/(mass*me_mp))
    
    return omega

def vib_period(omega):
    s_time_au = 2.418884326505e-17
    fs_s = 1e15
    period = 2*np.pi/omega*s_time_au*fs_s
    return period


def analyze_vib_props(pair_names, mass, struc_details, params_model):
    k = total_force_const(
        pair_names, struc_details['cn_pairs'],
        struc_details['dist_pairs'], params_model)
    omega = vib_freq(k, mass)
    period = vib_period(omega)
    
    vib_props = {}
    vib_props['k'] = k
    vib_props['omega'] = omega
    vib_props['period'] = period
    
    return vib_props
    
    


In [ ]:
# ang_bohr = 0.529177208
# eV_Ha = 27.211396
# me_mp = 1836
# 
# k_OO = force_const_term(cn_pairs_25['OO'], dist_pairs_25['OO'], params_O2000['OO'])
# omega_OO = vib_freq(k_OO, 16)
# 
# vib_time = vib_period(omega_OO)
# print(vib_time, ' fs')

In [ ]:
display(struc_BSE['135']['cn_pairs'])
display(struc_Brg['135']['cn_pairs'])

In [ ]:
moxy = 16

vib_props_Brg_O2000 = {}
print('==== 25 ====')
vib_props_Brg_O2000['25'] = analyze_vib_props(
    pair_names_Brg, moxy, struc_Brg['25'], params_O2000)

print('==== 135 ====')
vib_props_Brg_O2000['135'] = analyze_vib_props(
    pair_names_Brg, moxy, struc_Brg['135'], params_O2000)

print(vib_props_Brg_O2000['25']['period'], ' fs')
print(vib_props_Brg_O2000['135']['period'], ' fs')

In [ ]:
62/88

In [ ]:
import matplotlib.pyplot as plt
plt.scatter?

In [ ]:
moxy = 16

vib_props_BSE_O2000 = {}
print('==== 25 ====')
vib_props_BSE_O2000['25'] = analyze_vib_props(
    pair_names_Brg, moxy, struc_BSE['25'], params_O2000)

print('==== 135 ====')
vib_props_BSE_O2000['135'] = analyze_vib_props(
    pair_names_Brg, moxy, struc_BSE['135'], params_O2000)

print(vib_props_BSE_O2000['25']['period'], ' fs')
print(vib_props_BSE_O2000['135']['period'], ' fs')

In [ ]:
oxy_comp_Brg = struc_Brg['25']['comp']['O']

dSvib_25 = 3*oxy_comp_Brg*np.log(
    vib_props_Brg_O2000['25']['omega']/
    vib_props_BSE_O2000['25']['omega'])

dSvib_135 = 3*oxy_comp_Brg*np.log(
    vib_props_Brg_O2000['135']['omega']/
    vib_props_BSE_O2000['135']['omega'])


print('25 GPa: ', dSvib_25, ' kB')
print('135 GPa: ', dSvib_135, ' kB')

In [ ]:
m = (0.067-0.104)/(135-25)
b = 0.104 - 25*m
m*135 + b
print('m = ', m ,'; b = ', b)

In [ ]:
vib_props_Brg = {}
print('==== 25 ====')
vib_props_Brg['25'] = analyze_vib_props(
    pair_names_Brg, moxy, struc_Brg['25'], params_G2007)

print('\n==== 135 ====')
vib_props_Brg['135'] = analyze_vib_props(
    pair_names_Brg, moxy, struc_Brg['135'], params_G2007)



print(vib_props_Brg['25']['period'], ' fs')
print(vib_props_Brg['135']['period'], ' fs')


In [ ]:
vib_props_BSE = {}
print('==== 25 ====')
vib_props_BSE['25'] = analyze_vib_props(
    pair_names_BSE, moxy, struc_BSE['25'], params_G2007)

print('\n==== 135 ====')
vib_props_BSE['135'] = analyze_vib_props(
    pair_names_BSE, moxy, struc_BSE['135'], params_G2007)


print(vib_props_BSE['25']['period'], ' fs')
print(vib_props_BSE['135']['period'], ' fs')


In [ ]:
pair_names_BSE

In [ ]:
oxy_comp_Brg = struc_Brg['25']['comp']['O']

dSvib_25 = 3*oxy_comp_Brg*np.log(
    vib_props_Brg['25']['omega']/vib_props_BSE['25']['omega'])

dSvib_135 = 3*oxy_comp_Brg*np.log(
    vib_props_Brg['135']['omega']/vib_props_BSE['135']['omega'])

In [ ]:
print('25 GPa: ', dSvib_25, ' kB')
print('135 GPa: ', dSvib_135, ' kB')

In [ ]:
print('25 GPa: ', dSvib_25, ' kB')
print('135 GPa: ', dSvib_135, ' kB')

In [ ]:

vib_props_BSE_O2000 = {}

In [ ]:
oxy_comp_Brg

In [ ]:
3*0.6*np.log(omega_135/omega_25)